# Part 3: CQL syntax - examples
### Goals
* To create more complex CQL queries
* To understand how to make some specific queries in Alma
    * Search by library
    * Search for inventory in institution
    * Special operators
    * CQL limitations
   
### How to use SRU
#### SRU servers
* https://swisscovery.slsp.ch/view/sru/41SLSP_NETWORK?version=1.2&operation=explain
* https://renouvaud.primo.exlibrisgroup.com/view/sru/41BCULAUSA_NETWORK?version=1.2&operation=explain
* https://www.sudoc.abes.fr/cbs/sru/?operation=explain&version=1.1

#### Documentation
* https://slsp.ch/fr/metadonnees/
* https://developers.exlibrisgroup.com/alma/integrations/sru/
* https://knowledge.exlibrisgroup.com/Alma/Product_Documentation/Alma_Online_Help_(Francais)/130Int%C3%A9grations_avec_des_syst%C3%A8mes_externes/030Gestion_des_ressources/170Recherche_SRU%2F%2FSRW
* https://abes.fr/wp-content/uploads/2023/05/guide-utilisation-service-sru-catalogue-sudoc.pdf

In [3]:
import requests
from lxml import etree

parser = etree.XMLParser(remove_blank_text=True, remove_comments=True, ns_clean=True)

ns = {
    "srw": "http://www.loc.gov/zing/srw/",
    "marc": "http://www.loc.gov/MARC21/slim"
}

## Search by library
To search directly in holdings of the libraries, we need to use an SRU server at the IZ level.

Code of the libraries cannot be used in SRU, we need the ID of the library. We can see it in html of Primo or in response of API responses about library information.

In [5]:
params = {
    'version': '1.2',
    'operation': 'searchRetrieve',
    'query': 'alma.holding_Library=112062990005504 and alma.title=Arbre',
    'maximumRecords': '10'
}

url = 'https://swisscovery.ch/view/sru/41SLSP_UBS'

r = requests.get(url, params=params)
xml = etree.fromstring(r.content, parser=parser)

In [16]:
for record in xml.findall('.//marc:record', namespaces=ns):
    print('MMS ID: ', record.findtext('marc:controlfield[@tag="001"]', namespaces=ns))
    for ava_f in record.findall('.//marc:datafield[@tag="AVA"]', namespaces=ns):
        print('\t', ava_f.findtext('marc:subfield[@code="q"]', namespaces=ns), ' - ', ava_f.findtext('marc:subfield[@code="e"]', namespaces=ns))

MMS ID:  9961001430105504
	 Münchenstein – Bibliothek für Gestaltung Basel  -  available
MMS ID:  9951928380105504
	 Münchenstein – Bibliothek für Gestaltung Basel  -  available
MMS ID:  9941040010105504
	 Münchenstein – Bibliothek für Gestaltung Basel  -  available
MMS ID:  9910722950105504
	 Münchenstein – Bibliothek für Gestaltung Basel  -  available
MMS ID:  9967690830105504
	 Münchenstein – Bibliothek für Gestaltung Basel  -  available
MMS ID:  993360990105504
	 Münchenstein – Bibliothek für Gestaltung Basel  -  available
	 Basel - Staatsarchiv Basel-Stadt  -  available
	 Basel - UB Hauptbibliothek  -  available
	 Münchenstein – Bibliothek für Gestaltung Basel  -  available
	 Basel - UB Hauptbibliothek  -  unavailable


In [18]:
print(etree.tostring(ava_f, pretty_print=True, encoding='unicode'))

<datafield xmlns="http://www.loc.gov/MARC21/slim" ind1=" " ind2=" " tag="AVA">
  <subfield code="0">993360550105504</subfield>
  <subfield code="8">22243533230005504</subfield>
  <subfield code="a">41SLSP_UBS</subfield>
  <subfield code="b">A100</subfield>
  <subfield code="c">Magazin (&lt;a href="https://ub.unibas.ch/de/lieferfristen/"&gt;Lieferfristen beachten&lt;/a&gt;)</subfield>
  <subfield code="d">UBH</subfield>
  <subfield code="e">unavailable</subfield>
  <subfield code="f">1</subfield>
  <subfield code="g">1</subfield>
  <subfield code="v">from:1/8 until:1/8</subfield>
  <subfield code="j">MAG</subfield>
  <subfield code="p">5</subfield>
  <subfield code="q">Basel - UB Hauptbibliothek</subfield>
</datafield>



## Search for inventory in institutions

We use the network SRU server. Holdings of the institutions are present in the 852 datafield.

In [32]:
params = {
    'version': '1.2',
    'operation': 'searchRetrieve',
    'query': 'alma.title=Arbre',
    'maximumRecords': '20'
}

url = 'https://swisscovery.ch/view/sru/41SLSP_NETWORK'

r = requests.get(url, params=params)
xml = etree.fromstring(r.content, parser=parser)

In [33]:
for record in xml.findall('.//marc:record', namespaces=ns):
    mms_id = record.findtext('marc:controlfield[@tag="001"]', namespaces=ns)
    izs = []
    for f852 in record.findall('.//marc:datafield[@tag="852"]', namespaces=ns):
        
        iz = f852.findtext('marc:subfield[@code="a"]', namespaces=ns)
        if iz is not None:
            izs.append(iz)
    if len(izs) > 0:
        print(f'{mms_id}: {", ".join(izs)}')
    else:
        print(f'{mms_id}: IN NETWORK ONLY')

991117977839705501: 41SLSP_ETH
991032061729705501: 41SLSP_VGE
991006745929705501: 41SLSP_BCUFR, 41SLSP_VGE
991081713259705501: 41SLSP_VGE
991118204289705501: 41SLSP_BCUFR
991158803789705501: 41SLSP_VGE
991019524009705501: 41SLSP_UBS
991023388199705501: 41SLSP_IID
991170976881305501: 41SLSP_BCUFR
991159947589705501: 41SLSP_VGE
991013371239705501: 41SLSP_VGE
991110997119705501: 41SLSP_VGE, 41SLSP_HES
991121337109705501: 41SLSP_BCUFR
991171932911205501: 41SLSP_USI
991170709439905501: 41SLSP_UZB
991170866407605501: 41SLSP_VGE
991044659609705501: 41SLSP_VGE
991129431259705501: 41SLSP_HES
991157284479705501: 41SLSP_UZB
991170693657905501: 41SLSP_BCUFR


In [34]:
print(etree.tostring(f852, pretty_print=True, encoding='unicode'))

<datafield xmlns="http://www.loc.gov/MARC21/slim" ind1="0" ind2="1" tag="852">
  <subfield code="a">41SLSP_BCUFR</subfield>
  <subfield code="6">991018687998405509</subfield>
  <subfield code="9">P</subfield>
</datafield>



## Special syntax
* We need to use `==` when the item to search contains `-` or `_`, it means exact match
* We need to use `""` when item to search contains `<`, `>`, `=`, `/`, `(`, `)` and whitespace
* Indexes can cover more than one field

In [98]:
# With "=" we consider "-" and "_" as whitespaces, all words need to match
# With "==" the entire string should match
# Be sure to check the results afterwards...

params = {
    'version': '1.2',
    'operation': 'searchRetrieve',
#    'query': 'alma.title=="Les trios d\'anches de l\'Oiseau-lyre pour hautbois, clarinette et basson = The Oiseau-Lyre wind trios"', # 2 results
#    'query': 'alma.title="Les trios d\'anches de l\'Oiseau-lyre pour"', # 2 results
#    'query': 'alma.title=="Les trios d\'anches de l\'Oiseau-lyre pour"', # 0 result
#    'query': 'alma.title=="Les trios d\'anches de l\'Oiseau-lyre"', # 3 result due to 830a
    'query': 'alma.title="Les trios d\'anches de l\'Oiseau-lyre"', # 10 results
    'maximumRecords': '10'
}

url = 'https://swisscovery.ch/view/sru/41SLSP_NETWORK'

r = requests.get(url, params=params)
xml = etree.fromstring(r.content, parser=parser)
for record in xml.findall('.//marc:record', namespaces=ns):
    mms_id = record.findtext('marc:controlfield[@tag="001"]', namespaces=ns)
    title = record.findtext('marc:datafield[@tag="245"]/marc:subfield[@code="a"]', namespaces=ns)
    parent_title = record.findtext('marc:datafield[@tag="830"]/marc:subfield[@code="a"]', namespaces=ns)
    print(mms_id, title, subtitle, ' / ', parent_title)

991063504289705501 Cinq Divertissements [No. 4] pour hautbois, clarinette et basson = The Oiseau-Lyre wind trios  /  <<Les>> trios d'anches de l'Oiseau-Lyre
991040292589705501 Cinq divertissements pour hautbois, clarinette et basson = The Oiseau-Lyre wind trios  /  <<Les>> trios d'anches de l'Oiseau-lyre
991060703239705501 Cinq pièces en trio pour hautbois, clarinette et basson = The Oiseau-Lyre wind trios  /  <<Les>> trios d'anches de l'Oiseau-lyre
991138908959705501 Suite d'après Corrette pour hautbois, clarinette et basson = The Oiseau-Lyre wind trios  /  <<Les>> trios d'anches de l'Oiseau-lyre
991107883369705501 Suite d'après Corrette, op. 161 (1937) pour hautbois, clarinette et basson = The Oiseau-Lyre wind trios  /  <<Les>> trios d'anches de l'Oiseau-Lyre
991012476499705501 <<Les>> trios d'anches de l'Oiseau-lyre pour hautbois, clarinette et basson = The Oiseau-Lyre wind trios  /  None
991125184419705501 <<Les>> trios d'anches de l'oiseau-lyre pour hautbois, clarinette et basson 

In [71]:
params = {
    'version': '1.2',
    'operation': 'searchRetrieve',
    'query': 'alma.other_system_number="(swissbib)12675375*"',
    'maximumRecords': '10'
}

url = 'https://swisscovery.ch/view/sru/41SLSP_NETWORK'

r = requests.get(url, params=params)
xml = etree.fromstring(r.content, parser=parser)
for record in xml.findall('.//marc:record', namespaces=ns):
    mms_id = record.findtext('marc:controlfield[@tag="001"]', namespaces=ns)
    print(mms_id)

In [43]:
params = {
    'version': '1.2',
    'operation': 'searchRetrieve',
    'query': 'alma.other_system_number="(RERO)R009082738"',
    'maximumRecords': '1'
}

url = 'https://swisscovery.ch/view/sru/41SLSP_NETWORK'

r = requests.get(url, params=params)
print(r.text)

<?xml version="1.0" encoding="UTF-8" standalone="no"?><searchRetrieveResponse xmlns="http://www.loc.gov/zing/srw/">
  <version>1.2</version>
  <numberOfRecords>1</numberOfRecords>
  <records>
    <record>
      <recordSchema>marcxml</recordSchema>
      <recordPacking>xml</recordPacking>
      <recordData>
        <record xmlns="http://www.loc.gov/MARC21/slim">
          <leader>01777nam a2200337 c 4500</leader>
          <controlfield tag="001">991170638901605501</controlfield>
          <controlfield tag="005">20230407143840.0</controlfield>
          <controlfield tag="008">200924t20202017fr            00| | fre d</controlfield>
          <datafield ind1=" " ind2=" " tag="020">
            <subfield code="a">9782491674007</subfield>
          </datafield>
          <datafield ind1=" " ind2=" " tag="035">
            <subfield code="a">(swissbib)12675375X-41slsp_network</subfield>
          </datafield>
          <datafield ind1=" " ind2=" " tag="035">
            <subfield code="a">

## CQL limitations
In Alma, "not" boolean is not supported. In some indexes we can use "<>" instead of "not".